# pyoctree introduction
---
Requirements:
* pyoctree
* vtk >= 6.2.0

In [1]:
# Imports
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys, vtk
sys.path.append('../')
import pyoctree
from pyoctree import pyoctree as ot
import time
import random

In [2]:
print('pyoctree version = ', pyoctree.__version__)
print('vtk version = ', vtk.vtkVersion.GetVTKVersion())

pyoctree version =  0.2.4
vtk version =  8.1.0


## Load 3D model geometry (stl file)

In [3]:
# Read in stl file using vtk
reader = vtk.vtkSTLReader()
reader.SetFileName("knot.stl")
reader.MergingOn()
reader.Update()
stl = reader.GetOutput()
print("Number of points    = %d" % stl.GetNumberOfPoints())
print("Number of triangles = %d" % stl.GetNumberOfCells())

Number of points    = 38214
Number of triangles = 76428


In [4]:
# Extract polygon info from stl

# 1. Get array of point coordinates
numPoints   = stl.GetNumberOfPoints()
pointCoords = np.zeros((numPoints,3),dtype=float)
for i in range(numPoints):
    pointCoords[i,:] = stl.GetPoint(i)
    
# 2. Get polygon connectivity
numPolys     = stl.GetNumberOfCells()
connectivity = np.zeros((numPolys,3),dtype=np.int32)
for i in range(numPolys):
    atri = stl.GetCell(i)
    ids = atri.GetPointIds()
    for j in range(3):
        connectivity[i,j] = ids.GetId(j)

In [5]:
# Show format of pointCoords
pointCoords

array([[ 2.54317427,  7.41368246,  2.8599999 ],
       [ 2.51169538,  7.40434027,  2.8599999 ],
       [ 2.48648334,  7.48027468,  2.8599999 ],
       ..., 
       [ 1.20494771,  8.17073154,  2.8599999 ],
       [ 1.20671427,  8.17605209,  2.8599999 ],
       [ 1.20778596,  8.18155479,  2.8599999 ]])

In [6]:
# Show format of connectivity
connectivity

array([[    0,     1,     2],
       [    2,     3,     0],
       [    3,     4,     0],
       ..., 
       [38182, 38181, 38180],
       [38190, 38189, 38187],
       [38189, 38188, 38187]], dtype=int32)

## Generate octree

In [7]:
# Create octree structure containing stl poly mesh
max_depth = 5
#tree = ot.PyOctree(pointCoords,connectivity, int max_point_per_voxel, int max_octree_depht)
t=time.time()
tree = ot.PyOctree(pointCoords,connectivity,100,max_depth)
print(time.time()-t)

0.9311907291412354


In [8]:
# Print out basic Octree data
print("Size of Octree               = %.3fmm" % tree.root.size)
print("Number of Octnodes in Octree = %d" % tree.getNumberOfNodes())
print("Number of polys in Octree    = %d" % tree.numPolys)

Size of Octree               = 3.115mm
Number of Octnodes in Octree = 1753
Number of polys in Octree    = 76428


# Separate inside cells from the outside one

In [9]:
print(tree.getNumberOfNodes())
print(tree.getNumberOfLeafs())
        

1753
1534


In [10]:
a=tree.getLeafs()
len(a)
n=0
for i in a:
    if i.isLeaf:
        n+=1
print(n)

1534


In [40]:
b = tree.getNodes()
print(len(b))
for i in range(20):
    print(b[i].numPolys)
print(type(b[0]))

1753
0
0
0
0
0
0
0
0
0
0
24
0
0
0
0
0
195
0
0
0
<class 'pyoctree.pyoctree.PyOctnode'>


# For each node check wheter it belongs to the mesh or not

In [12]:
type(tree.getNodes()[0])

pyoctree.pyoctree.PyOctnode

In [35]:
b[19].isInside = True
print(b[19].isInside)
print(b[19].position)

True
[ 1.4077959   6.83933439  2.80578265]


In [41]:
r = tree.root.size
for node in tree.getNodes():
    if node.numPolys == 0:
        coords = node.position
        ray = np.array([[coords[0],coords[1],coords[2]+r],[coords[0],coords[1],coords[2]-r]],dtype=np.float32)
        if len(tree.rayIntersection(ray))%2:
            node.isInside = True
    else:
        node.isInside = True
        


In [42]:
n=0
m=0
for node in tree.getNodes():
    m+=1
    if node.isInside:
        n+=1
print(m)
print(n)

1753
1026


In [15]:
# Get intersection points for a single ray



xs,xe,ys,ye,zs,ze = stl.GetBounds()
x = 0.5*np.mean([xs,xe])
y = np.mean([ys,ye])
rayPointList = np.array([[[x,y,zs],[x,y,ze]]],dtype=np.float32)

# Find if an intersection occurred
for i in tree.rayIntersections(rayPointList):
    print(i==1)

# Get intersection points for a single ray
ray = rayPointList[0]
for i in tree.rayIntersection(ray):
    print('Intersection coords = [%.2f, %.2f, %.2f]' % tuple(i.p), ',  Parametric. dist. along ray = %.2f' % i.s)ction coords = [%.2f, %.2f, %.2f]' % tuple(i.p), ',  Parametric. dist. along ray = %.2f' % i.s)

SyntaxError: invalid syntax (<ipython-input-15-a7b5924b00ac>, line 17)

## Explore tree nodes

In [16]:
# Get the root node
print(tree.root)

<PyOctnode, Id: 0, isLeaf: False, numPolys: 0>


In [17]:
# Get the branches of the root node, OctNode 0
tree.root.branches

[<PyOctnode 0-0>,
 <PyOctnode 0-1>,
 <PyOctnode 0-2>,
 <PyOctnode 0-3>,
 <PyOctnode 0-4>,
 <PyOctnode 0-5>,
 <PyOctnode 0-6>,
 <PyOctnode 0-7>]

In [18]:
# Get OctNode 0-0 (first branch of the root node)
print(tree.root.branches[0])

<PyOctnode, Id: 0-0, isLeaf: False, numPolys: 0>


In [19]:
# Get the branches of OctNode 0-0 (first branch of the root node) using getNodeFromId function
for i in range(8):
    v_id = '0-0-'+str(i)
    print(tree.getNodeFromId(v_id).level)
print(tree.root.position)
print(tree.root.size)

3
3
3
3
3
3
3
3
[ 1.50515477  7.32612872  3.68201244]
3.1154836893081668


In [20]:
# An octnode that has no branches is a leaf
print('OctNode 0-0-1 is a leaf = ', tree.getNodeFromId('0-3-1').isLeaf)
print(tree.getNodeFromId('0-3-1'))

OctNode 0-0-1 is a leaf =  False
<PyOctnode, Id: 0-3-1, isLeaf: False, numPolys: 0>


## Explore polygons in tree

In [21]:
# Get list of first 10 polygons stored in tree
tree.polyList[0:10]

[<PyTri 0>,
 <PyTri 1>,
 <PyTri 2>,
 <PyTri 3>,
 <PyTri 4>,
 <PyTri 5>,
 <PyTri 6>,
 <PyTri 7>,
 <PyTri 8>,
 <PyTri 9>]

In [22]:
# Get details of the first tri in the tree
tri = tree.polyList[0]
s = []
for i in range(3):
    s.append('[%.3f, %.3f, %.3f]' % tuple(tri.vertices[i][:3]))
s = ','.join(s)

print('Tri label = %d' % tri.label)
print('Vertex coordinates = %s' % s)
print('Face normal direction = [%.2f, %.2f, %.2f]' % tuple(tri.N))
print('Perp. distance from origin = %.3f' % tri.D)

Tri label = 0
Vertex coordinates = [2.543, 7.414, 2.860],[2.512, 7.404, 2.860],[2.486, 7.480, 2.860]
Face normal direction = [0.00, 0.00, -1.00]
Perp. distance from origin = -2.860


In [23]:
# Find the OctNode that the tri with label=0 lies within
tree.getNodesFromLabel(0)

<PyOctnode 0-3-1-4-5>

In [24]:
# Test if OctNode contains a particular tri label
tree.getNodeFromId('0-3-1-4-5-4').hasPolyLabel(0)

AttributeError: 'NoneType' object has no attribute 'hasPolyLabel'

In [25]:
#tree.getNodeFromId('0-3-1-4-5-4').get_data2()

## Find intersections between 3D object and ray

In [30]:
# Create a list containing a single ray
xs,xe,ys,ye,zs,ze = stl.GetBounds()
print(stl.GetBounds())
x = 0.5*np.mean([xs,xe])
print('x =',x)
y = np.mean([ys,ye])
print('y =',y)
rayPointList = np.array([[[x,y,zs],[x,y,ze]]],dtype=np.float32)
print(rayPointList)

(0.025677038356661797, 2.9846324920654297, 5.842565059661865, 8.8096923828125, 2.859999895095825, 4.504024982452393)
x = 0.752577382606
y = 7.32612872124
[[[ 0.75257736  7.32612896  2.8599999 ]
  [ 0.75257736  7.32612896  4.50402498]]]


In [31]:
# Find if an intersection occurred
for i in tree.rayIntersections(rayPointList):
    print(i==1)
print(tree.rayIntersections(rayPointList))

True
[1]


In [34]:
# Get intersection points for a single ray
ray = rayPointList[0]
print(ray)
for i in tree.rayIntersection(ray):
    print('Intersection coords = [%.2f, %.2f, %.2f]' % tuple(i.p), ',  Parametric. dist. along ray = %.2f' % i.s)
print(len(tree.rayIntersection(ray))%2)

[[ 0.75257736  7.32612896  2.8599999 ]
 [ 0.75257736  7.32612896  4.50402498]]
Intersection coords = [0.75, 7.33, 3.32] ,  Parametric. dist. along ray = 0.46
Intersection coords = [0.75, 7.33, 3.64] ,  Parametric. dist. along ray = 0.78
Intersection coords = [0.75, 7.33, 3.87] ,  Parametric. dist. along ray = 1.01
Intersection coords = [0.75, 7.33, 4.42] ,  Parametric. dist. along ray = 1.56
0


In [ ]:
for 

## Write out a vtk file of the octree structure 

In [ ]:
# Create a vtk representation of Octree
tree.getOctreeRep()